In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")

# Entering the APIs to be Sotred

In [ ]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")



# Call the functions to set the API key and credentials
set_ngrok_api_key()

# [](http://)[](http://)******Fucntions for the Operations of CNN

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt
import requests
import base64
import io

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)



# Additional Kernels
kernels = {
    "Vertical Edge": np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    "Horizontal Edge": np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    "Smoothing": np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9,
    "Gaussian Blur": np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]]) / 16,
    "Emboss": np.array([[-2, -1, 0], [-1, 1, 1], [0, 1, 2]])
}

def apply_kernels_with_multiple_layers(images, kernels_list):
    """
    Apply multiple convolutional layers with different kernels sequentially to the images.
    """
    input_layer = Input(shape=(500, 500, 1))
    x = input_layer

    # Add convolutional layers for each kernel in the list
    for kernel in kernels_list:
        kernel = kernel[..., np.newaxis, np.newaxis]  # Adjust kernel dimensions
        x = Conv2D(
            filters=1,
            kernel_size=(3, 3),
            kernel_initializer=tf.keras.initializers.Constant(kernel),
            use_bias=False,
            padding='same'
        )(x)

    # Build and compile the model
    model = Model(inputs=input_layer, outputs=x)
    model.compile(optimizer='adam', loss='mse')

    # Predict and return the output
    return model.predict(images)


def load_and_process_image(image_content):
    """Load the image from bytes and process it."""
    try:
        # Load image using OpenCV
        img = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Invalid image data")

        # Resize image
        img = cv2.resize(img, (500, 500))
        img = img[np.newaxis, ..., np.newaxis] / 255.0  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error loading and processing image: {e}")
        return None



@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Download the image
        print(f"Downloading file from: {file_url}")
        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Load and process the image
        image = load_and_process_image(file_response.content)
        if image is None:
            return jsonify({"error": "Failed to load image"}), 400

        # Apply kernels with multiple convolutional layers
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)
        results = {}

        for name, kernel in kernels.items():
            print(f"Applying multiple layers with {name} kernel...")
            convolved_image = apply_kernels_with_multiple_layers(image, [kernel] * 2)  # Using the same kernel for two layers
            
            # Save image as PNG file
            output_path = os.path.join(output_dir, f"{name}_result.png")
            plt.imsave(output_path, convolved_image.squeeze(), cmap='gray')
            
            # Read image and encode it to base64
            with open(output_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

            # Store results with the image in base64 and additional metadata
            results[name] = {
                "image": img_base64,
                "kernelName": name,
                "isMultipleLayers": True
            }

        print("Image processing completed.")
        return jsonify({
            "message": "Image processing completed",
            "results": results
        }), 200

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500



# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)


# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url


# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Start ngrok and print the URL
public_url = run_ngrok()

# Keep the program running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Server shutting down.")


In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt
import requests
import base64

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Kernels for convolution
kernels = {
    "Vertical Edge": np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]]),
    "Horizontal Edge": np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]]),
    "Sharpen": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    "Smoothing": np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9
}

# Test endpoint to check if the server is running
@app.route('/test', methods=['POST'])
def test():
    return jsonify({"message": "Flask server is running!"})


def load_and_process_image(image_content):
    """Load the image from bytes and process it."""
    try:
        # Load image using OpenCV
        img = cv2.imdecode(np.frombuffer(image_content, np.uint8), cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("Invalid image data")

        # Resize image
        img = cv2.resize(img, (500, 500))
        img = img[np.newaxis, ..., np.newaxis] / 255.0  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error loading and processing image: {e}")
        return None


def apply_kernel(images, kernel):
    """Apply the given kernel to the image."""
    kernel = kernel[..., np.newaxis, np.newaxis]
    input_layer = Input(shape=(500, 500, 1))
    conv_layer = Conv2D(
        filters=1,
        kernel_size=(3, 3),
        kernel_initializer=tf.keras.initializers.Constant(kernel),
        use_bias=False,
        padding='same'
    )(input_layer)
    model = Model(inputs=input_layer, outputs=conv_layer)
    model.compile(optimizer='adam', loss='mse')
    return model.predict(images)


@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Download the image
        print(f"Downloading file from: {file_url}")
        file_response = requests.get(file_url, stream=True)
        file_response.raise_for_status()

        # Load and process the image
        image = load_and_process_image(file_response.content)
        if image is None:
            return jsonify({"error": "Failed to load image"}), 400

        # Apply kernels and save results
        output_dir = "output"
        os.makedirs(output_dir, exist_ok=True)
        results = {}
        for name, kernel in kernels.items():
            print(f"Applying {name} kernel...")
            convolved_image = apply_kernel(image, kernel)
            output_path = os.path.join(output_dir, f"{name}_result.png")
            plt.imsave(output_path, convolved_image.squeeze(), cmap='gray')

            # Read image and encode it to base64
            with open(output_path, "rb") as img_file:
                img_base64 = base64.b64encode(img_file.read()).decode('utf-8')

            # Store results with the image in base64 and additional metadata
            results[name] = {
                "image": img_base64,
                "kernelName": name,
                "isMultipleLayers": False  # Added flag for single-layer kernel
            }

        print("Image processing completed.")
        return jsonify({
            "message": "Image processing completed",
            "results": results
        }), 200

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500


# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)


# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url


# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Start ngrok and print the URL
public_url = run_ngrok()

# Keep the program running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Server shutting down.")


 * Serving Flask app '__main__'
 * Debug mode: off
Ngrok public URL: NgrokTunnel: "https://8305-35-203-133-124.ngrok-free.app" -> "http://localhost:5000"
Applying Vertical Edge kernel...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step
Applying Horizontal Edge kernel...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Applying Sharpen kernel...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Applying Smoothing kernel...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image processing completed.


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D
import matplotlib.pyplot as plt

def download_image(image_url, download_path):
    """
    Downloads an image from a URL and saves it to the specified path.
    """
    import requests
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        with open(download_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Image downloaded successfully: {download_path}")
        return download_path
    except Exception as e:
        print(f"Error downloading image from {image_url}: {e}")
        return None

def load_images_from_folder(image_folder, resize_shape=(500, 500), grayscale=True):
    """
    Loads images from a specified folder, resizes them, and normalizes pixel values.
    """
    images = []
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, resize_shape)
            images.append(img)
        else:
            print(f"Warning: Unable to load image {img_path}")
    return np.array(images)

def apply_cnn_kernel(images, kernel):
    """
    Applies a convolutional kernel to a set of images using a CNN.
    """
    kernel = kernel[..., np.newaxis, np.newaxis]  # Shape becomes (3, 3, 1, 1)

    input_layer = Input(shape=(500, 500, 1))
    conv_layer = Conv2D(
        filters=1,
        kernel_size=(3, 3),
        kernel_initializer=tf.keras.initializers.Constant(kernel),
        use_bias=False,
        padding='same'
    )(input_layer)
    model = Model(inputs=input_layer, outputs=conv_layer)
    model.compile(optimizer='adam', loss='mse')

    return model.predict(images)

def save_processed_images(images, output_dir, kernel_name):
    """
    Saves processed images to the specified output directory.
    """
    os.makedirs(output_dir, exist_ok=True)
    for i, img in enumerate(images):
        output_path = os.path.join(output_dir, f'{kernel_name}_img{i+1}.png')
        plt.imsave(output_path, img.squeeze(), cmap='gray')
        print(f"Saved: {output_path}")

def process_image_with_kernels(image_paths, kernels, output_dir):
    """
    Processes images with multiple CNN kernels and saves the results.
    """
    # Load images
    images = [cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in image_paths]
    images = [cv2.resize(img, (500, 500)) for img in images if img is not None]
    if not images:
        raise ValueError("No valid images loaded from provided paths.")
    
    images = np.array(images)[..., np.newaxis] / 255.0  # Normalize and add channel

    # Apply kernels
    for name, kernel in kernels.items():
        print(f"Applying {name} kernel...")
        convolved_images = apply_cnn_kernel(images, kernel)
        save_processed_images(convolved_images, output_dir, name)
        print(f"{name} kernel applied and saved successfully.")


# Implementing the Flask API GateWays

# Main part for running the API gateWay

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from PIL import Image  # For image preview (optional)
from io import BytesIO  # For handling file in memory

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Test endpoint to check if the server is running
@app.route('/test', methods=['POST'])
def test():
    return jsonify({"message": "Flask server is running!"})


import requests  # Import the requests library for making HTTP requests

@app.route('/process-url', methods=['POST'])
def process_url():
    try:
        # Get data from the request
        data = request.get_json()
        if not data or 'fileUrl' not in data or 'ngrokUrl' not in data:
            return jsonify({"error": "Missing required fields 'fileUrl' or 'ngrokUrl'"}), 400

        file_url = data['fileUrl']
        ngrok_url = data['ngrokUrl']

        # Log the received data
        print(f"Received fileUrl: {file_url}")
        print(f"Received ngrokUrl: {ngrok_url}")

        # Download and preview the file
        def download_and_preview(url):
            try:
                print(f"Downloading file from: {url}")
                file_response = requests.get(url, stream=True)
                file_response.raise_for_status()  # Ensure the request was successful
                
                # Read the file in memory
                file_content = BytesIO(file_response.content)

                # Try to preview if it's an image
                try:
                    img = Image.open(file_content)
                    img.show()  # Opens the image in the default viewer
                    print(f"Image preview: {img.format}, {img.size}, {img.mode}")
                except Exception as e:
                    print(f"File is not an image: {e}")

                return "File downloaded and previewed successfully"
            except Exception as e:
                print(f"Error downloading or previewing file: {e}")
                return "Error during file preview"

        # Call the download and preview function
        preview_result = download_and_preview(file_url)
        print(preview_result)

        # Prepare the payload for the POST request
        payload = {
            "fileUrl": file_url,
            "ngrokUrl": ngrok_url
        }

        # Send the POST request to the external endpoint
        external_endpoint = f"{ngrok_url}/api/v2/post-image-details"  # Construct the URL
        response = requests.post(external_endpoint, json=payload)

        # Check if the response was successful
        if response.status_code == 200:
            print(f"POST request successful: {response.json()}")
            return jsonify({
                "message": "POST request sent successfully",
                "response": response.json(),
                "previewResult": preview_result
            }), 200
        else:
            print(f"POST request failed with status {response.status_code}: {response.text}")
            return jsonify({"error": "POST request failed", "details": response.text}), response.status_code

    except Exception as e:
        print(f"Error processing request: {e}")
        return jsonify({"error": "An error occurred while processing the request"}), 500


# Function to run Flask server
def run_flask():
    app.run(host="0.0.0.0", port=5000)  # Running on port 5000

# Function to start ngrok and expose the Flask server
def run_ngrok():
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    # Open a ngrok tunnel to the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

# Start Flask server in a separate thread to keep the program running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Start ngrok and print the URL where Flask is accessible
public_url = run_ngrok()

# Keep the program running to maintain the server
try:
    while True:
        time.sleep(1)  # Keeps the program running
except KeyboardInterrupt:
    print("Server shutting down.")
